In [369]:
from transformers import AutoTokenizer

In [370]:
cp = "Tables"
tok = AutoTokenizer.from_pretrained(cp)

In [371]:
inp = tok(["je voudrais avoir la liste des commandes fournisseurs est ce que ça change ?.","ceci va changer pas mal de choses"])
print(inp)


{'input_ids': [[8510, 3979, 6282, 386, 4263, 226, 2943, 287, 3657, 5703, 2776, 4265, 4666, 30, 59, 2058, 169, 24, 10], [4265, 382, 6144, 15258, 775, 5207, 107, 12233, 80]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [372]:
inp = tok(["comment ajouter la propriété [MASK] de type [MASK].","je voudrais ajouter la propriété Nom de l'utilisateur de type string."])
print(inp)


{'input_ids': [[17475, 17156, 67, 226, 8254, 4, 107, 265, 4, 10], [8510, 3979, 6282, 386, 17156, 67, 226, 8254, 455, 107, 39, 6, 1159, 107, 265, 7934, 10]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [373]:
def tok2(inputs):
    result = []
    a = inputs.input_ids[0]
    b = inputs.input_ids[1]
    cnt = 0
    mask = -1
    append = False;
    print(a)
    for e in b:
        if e==a[cnt]:
            cnt = cnt + 1
            append = False
        if a[cnt]==4:
            cnt = cnt + 1
            mask = mask + 1
            result.append([])
            append = True
        if append:
            result[mask].append(e)
        print(e, a[cnt], cnt, tok.decode(result))
        if cnt>=len(a):
            break;
        
    print(cnt)
    print(tok.decode(result))

In [374]:
tok2(inp)

[17475, 17156, 67, 226, 8254, 4, 107, 265, 4, 10]
8510 17475 0 
3979 17475 0 
6282 17475 0 
386 17475 0 
17156 17475 0 
67 17475 0 
226 17475 0 
8254 17475 0 
455 17475 0 
107 17475 0 
39 17475 0 
6 17475 0 
1159 17475 0 
107 17475 0 
265 17475 0 
7934 17475 0 
10 17475 0 
0



In [375]:
inp = tok(["* du [MASK] au [MASK].","du premier juin au 15 juillet."])
print(inp)

{'input_ids': [[0, 215, 4, 635, 4, 10], [215, 7769, 37, 105, 84, 635, 695, 37, 6374, 56, 10]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [376]:
def split4(inputs):
    result = []
    current = []
    a = inputs.input_ids[0][1:]
    for e in a:
        if e != 4 : 
            current.append(e)
        else:
            result.append(current)
            current = []
    result.append(current)
    return result

In [377]:
split4(inp)[0]

[215]

In [378]:
def _search(inputs,splits,idx):
    result = []
    current = []
    deb = -1
    fin = -1
    i = 0
    cnt = 0
    cntLoss = 0
    # print(inputs)
    toSearch = splits[idx]
    # print(toSearch)
    b = inputs.input_ids[1]
    for e in b:
        # print(e,cnt)
        if deb == -1 :
            if e==toSearch[cnt]:
                cnt = cnt + 1
                deb = i
                cntLoss = 0 
                if len(toSearch) == 1:
                    fin = i+1
                    cnt = 0
                    cntLoss = 0
                    #print("found")
                    break
        else:
            if e==toSearch[cnt]:
                cnt = cnt + 1
                cntLoss = 0 
                if cnt ==len(toSearch):
                    fin = i+1
                    #print("found")
                    break
            else:
                cntLoss = cntLoss + 1
                if cntLoss>2:
                    deb = -1
                    cnt = 0
        i = i +1
    #print("------")
    #print(deb)    
    #print(fin)
    #for i in range(deb,fin+1):
    return [deb,fin]
    #print(tok.decode(b[deb:fin]).replace(" ##",""), ": ", tok.decode(b[8:13]).replace(" ##","").replace(" ' ","'"))

In [379]:
_search(inp,split4(inp),1)

[5, 6]

In [417]:
class searchResult:
    def __init__(self):
        self.start = 0
        self.end = 0
        self.txt = ''
        self.extract = ''
        
def search(inputs):
    print(inputs)
    split = split4(inputs)
    result = []
    first = True
    for i in range(len(split)):
        lst = _search(inputs,split,i)
        if i != len(split)-1:
            result.append(searchResult())
            result[i].start = lst[1]
        if i !=0:
            result[i-1].end = lst[0]
    for i in range(len(split)-1):
        result[i].extract = tok.decode(inputs.input_ids[1][result[i].start:result[i].end]).replace(" ##","")
        result[i].txt = tok.decode(split[i]).replace(" ##","")

    found = True
    for r in result:
        if r.start == -1:
            found = False
            break
        if r.end == -1:
            found = False
            break
       
    return found, result   
    #for r in result:
    #    print(r.__dict__)
        

In [418]:
f, res = search(tok(["* du [MASK] au [MASK].","je voudrais réserver du premier juillet au 15 juillet."]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")
#a=searchResult()

{'input_ids': [[0, 215, 4, 635, 4, 10], [8510, 3979, 6282, 386, 3860, 126, 215, 7769, 37, 6374, 56, 635, 695, 37, 6374, 56, 10]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
du  :  premier juillet 7 11
au  :  15 juillet 12 16


In [382]:
f, res = search(tok(["comment ajouter la propriété [MASK] de type [MASK].","je voudrais ajouter la propriété Nom de l'utilisateur de type string."]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

ajouter la propriete  :  nom de l ' utilisateur 8 13
de type  :  string 15 16


In [383]:
f, res = search(tok(["comment ajouter une constante [MASK] avec comme valeur [MASK]µ","je voudrais ajouter la constante fichier ini avec comme valeur Yousign.ini µ"]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

not found


In [404]:
f, res = search(tok(["comment ajouter la constante [MASK] avec comme valeur [MASK].","je voudrais ajouter la propriété Nom de l'utilisateur de type string."]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

not found


In [405]:
f, res = search(tok(["comment ajouter la propriété [MASK] de type [MASK].","je voudrais ajouter la constante fichier ini avec comme valeur Yousign.ini."]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

not found


In [406]:
tok(["comment ajouter la constante [MASK] avec comme valeur [MASK]."]).__dict__

{'data': {'input_ids': [[17475,
    17156,
    67,
    226,
    17535,
    73,
    4,
    1531,
    17474,
    195,
    4,
    10]],
  'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
 '_encodings': [Encoding(num_tokens=12, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 '_n_sequences': 1}

In [415]:
gCmds = []
def addCmd(question, params,nomCommande, where):
    gCmds.append([question, params,nomCommande, where])

addCmd(tok("comment ajouter la constante [MASK] avec comme valeur [MASK].").input_ids,["ajout_constante"],tok("ajoutcode").input_ids, tok("constantes").input_ids)
addCmd(tok("comment ajouter la constante [MASK] avec comme valeur [MASK].").input_ids,["ajout constante"],tok("ajoutcode").input_ids, tok("constantes").input_ids)

addCmd(tok("comment ajouter la propriété [MASK] de type [MASK].").input_ids,[],tok("exemplecode").input_ids, tok("public").input_ids)
addCmd(tok("comment créer une unité ayant pour nom [MASK].").input_ids,[],tok("exemplecode").input_ids, tok("nouveau").input_ids)
addCmd(tok("comment créer une unité avec pour nom [MASK].").input_ids,[],tok("exemplecode").input_ids, tok("nouveau").input_ids)
addCmd(tok("comment créer une unité de nom [MASK].").input_ids,[],tok("exemplecode").input_ids, tok("nouveau").input_ids)
addCmd(tok("ayant pour nom [MASK].").input_ids,[],tok("memoire").input_ids, tok("nom").input_ids)
addCmd(tok("le nom [MASK].").input_ids,[],tok("memoire").input_ids, tok("nom").input_ids)

In [416]:
gCmds

[[[17475, 17156, 67, 226, 17535, 73, 4, 1531, 17474, 195, 4, 10],
  ['ajout_constante'],
  [7917, 936],
  [17535, 186]],
 [[17475, 17156, 67, 226, 17535, 73, 4, 1531, 17474, 195, 4, 10],
  ['ajout constante'],
  [7917, 936],
  [17535, 186]],
 [[17475, 17156, 67, 226, 8254, 4, 107, 265, 4, 10], [], [8165, 936], [8047]],
 [[17475, 13335, 2244, 1797, 4618, 225, 455, 4, 10], [], [8165, 936], [3326]],
 [[17475, 13335, 2244, 1797, 1531, 225, 455, 4, 10], [], [8165, 936], [3326]],
 [[17475, 13335, 2244, 1797, 107, 455, 4, 10], [], [8165, 936], [3326]],
 [[4618, 225, 455, 4, 10], [], [1902, 321], [455]],
 [[467, 455, 4, 10], [], [1902, 321], [455]]]

In [409]:
f, res = search(tok(["* comme valeur [MASK][SEP]","je voudrais ajouter la constante ayant pour nom fichier ini. avec comme valeur Yousign.ini[SEP]"]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

comme valeur  :  yousign . ini 18 24


In [410]:
f, res = search(tok(["* avec pour nom [MASK],","je voudrais ajouter la constante avec pour nom fichier ini, avec comme valeur Yousign.ini µ"]))
#print(res)
if f:
    for r in res:
        print(r.txt," : ",r.extract, r.start,r.end)
else:
    print("not found")

avec pour nom  :  fichier ini 12 14


In [414]:
tok("le nom de la constante qui a pour valeur [Valeur][SEP]")

{'input_ids': [467, 455, 107, 226, 17535, 73, 1546, 28, 225, 195, 25, 195, 26, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [412]:
tok("quel est ")

{'input_ids': [4666, 66, 2776], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}